In [1]:
import numpy as np
import keras.datasets
%matplotlib inline
import matplotlib.pyplot as plt
import os
import keras

/home/mara/venv/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
class MNISTRandom():
    '''
    Params
    corrupted: float
      Default 0.0
    num_classes: int
      Default 10.
    '''
    
    def __init__(self, label_corrupt_p=0.0, gaussian_noise_f = 0.0, num_classes=10, **kwargs):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = keras.datasets.mnist.load_data()
        self.num_classes = num_classes
        # note: corruption is performed on the training set. 
        # you test on real data to check generalization
        if label_corrupt_p > 0.0:
            self.label_corrupt(label_corrupt_p)
        if gaussian_noise_f > 0.0:
            self.gaussian_noise(gaussian_noise_f)
    
    def old_label_corrupt(self, corrupted):
        # Corrupts the labels in the training set according to
        # the specified corruption probability
        labels=np.array(self.y_train)
        #labels = np.reshape(len(labels),1)
        np.random.seed(1)
        mask = np.random.rand(len(labels)) <= corrupted
        rnd_labels = np.random.choice(self.num_classes, mask.sum())
        #rnd_labels = np.reshape(rnd_labels, (len(labels),1))
        labels[mask] = rnd_labels
        labels = [int(x) for x in labels]
        # corruption
        self.y_train = labels
        
    def label_corrupt(self, corrupted):
        # Corrupts the labels in the training set according to
        # the specified corruption probability
        labels=np.array(self.y_train)
        #labels = np.reshape(len(labels),1)
        np.random.seed(1)
        mask = np.random.rand(len(labels)) <= corrupted
        #rnd_labels = np.random.choice(self.num_classes, mask.sum())
        true_labels = labels[mask]
        #rnd_labels = np.reshape(rnd_labels, (len(rnd_labels),1))
        #labels[mask] = rnd_labels
        np.random.shuffle(true_labels)
        labels[mask] = true_labels
        #labels = [int(x) for x in labels]
        # corruption
        self.y_train = labels
    def gaussian_noise(self, gaussian_noise_f):
        # Adds Gaussian Noise to the images,
        # matching the real dataset's mean and variance
        data = np.array(self.x_train)
        mean = np.mean(data)
        var = np.std(data)
        sigma = var**0.5
        #import pdb; pdb.set_trace()
        n_samples, row, col = data.shape
        mask = np.random.rand(n_samples) <= gaussian_noise_f
        gaussian = np.random.normal(mean, sigma, (row, col))
        gaussian = gaussian.reshape(row, col)
        noisy_imgs = [x+gaussian for x in data[mask]]
        data[mask] = noisy_imgs
        self.x_train = data

class MLP():
    '''
    Multilayer Perceptron for experiments on MNIST
    
    Params
    deep: int
      Default 2
    wide: int
      Default 512
    optimizer: string
      Default SGD
    lr: float
      Default 1e-2
    epochs: int
      Default 10
    batch_size: int
      Default: 32
    input_shape: int
      Default 28
    n_classes: int
      Default 10
    '''
    
    def __init__(self, deep=2, wide=512, optimizer='SGD', lr=1e-2, epochs=10, 
                 batch_size=32, input_shape=28, n_classes=10, **kwargs):
        
        #mask_shape = np.ones((1,512))
        #mask = keras.backend.variable(mask_shape)

        mlp = keras.models.Sequential()
        mlp.add(keras.layers.Flatten(input_shape=(input_shape,input_shape)))
        counter = 0
        while counter<deep:
            mlp.add(keras.layers.Dense(wide, activation=keras.layers.Activation('relu')))
            counter+=1
        loss_function = 'categorical_crossentropy'
        activation = 'softmax'
        if n_classes == 2:
            loss_function = 'binary_crossentropy'
            activation = 'sigmoid'
        mlp.add(keras.layers.Dense(n_classes, activation=keras.layers.Activation(activation)))

        #masking_layer = keras.layers.Lambda(lambda x: x*mask)(bmlp.layers[-2].output)
        #if n_hidden_layers>1:
        #    while n_hidden_layers!=1:
        #        masking_layer= keras.layers.Dense(512, activation=keras.layers.Activation('sigmoid'))(masking_layer)
        #        n_hidden_layers-=1
        #decision_layer = keras.layers.Dense(10, activation=keras.layers.Activation('softmax'))(masking_layer)
        #masked_model = keras.models.Model(input= bmlp.input, output=decision_layer)
        model = keras.models.Model(input=mlp.input, output=mlp.output)
        model.compile(optimizer=optimizer,
                      loss=loss_function,
                      metrics=['accuracy'])
        self.model = model
        self.epochs = epochs
        self.batch_size = batch_size
        
    
    def train(self, dataset):
        #import pdb; pdb.set_trace()
        x_train = dataset.x_train
        y_train = dataset.y_train
        x_train = x_train / 255.0

        try:
            shape1, shape2 = y_train.shape()
        except:
            y_train = keras.utils.to_categorical(y_train)
        
        history=self.model.fit(x_train, y_train, epochs=self.epochs, batch_size=self.batch_size, validation_split=0.1)
        self.training_history=history
        
    def save(self, name, folder):
        try:
            os.listdir(folder)
        except:
            os.mkdir(folder)
            
        #model_json = self.model.to_json()
        #with open(folder+"/"+name+".json", "w") as json_file:
        #    json_file.write(model_json) 
        # serialize weights to HDF5
        self.model.save_weights(folder+"/"+name+".h5")
        print("Saved model to disk")
        np.save(folder+'/'+name+'_history', self.training_history.history)

In [3]:
mnist = MNISTRandom()
depths = [6]
widths = [4,16,64,256, 1024, 2048, 4096]
#widths = [4096]
#label_corrupt_pbs = [0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
label_corrupt_pbs = [ 0.2, 0.4, 0.6, 0.8, 1.0]
#gaussian_noise_fs = [0., 0.2, 0.4, 0.6, 0.8, 1.0]
try:
    del mlp
except:
    pass

for d in depths:
    for w in widths:
        for lcp in label_corrupt_pbs[:]:
            try:
                del mlp
            except:
                pass
            try:
                del mnist
            except:
                pass
            print 'DEEPTH: {}, WIDTH: {}, LCP: {}'.format(d,w,lcp)
            mnist = MNISTRandom(label_corrupt_p = lcp)
            mlp = MLP(deep=d, wide=w,epochs=300)
            mlp.train(mnist)
            mlp.save('mnist_{}d_{}w_{}lcp'.format(d,w,lcp), 'mnist_exps_new_corruption')

for d in depths:
    for w in widths:
        for gnf in gaussian_noise_fs:
            try:
                del mlp
            except:
                pass
            try:
                del mnist
            except:
                pass
            print 'DEEPTH: {}, WIDTH: {}, GNF: {}'.format(d,w,gnf)
            mnist = MNISTRandom(gaussian_noise_f=gnf)
            mlp = MLP(deep=d, wide=w,epochs=300)
            mlp.train(mnist)
            mlp.save('mnist_{}d_{}w_{}gnf'.format(d,w,gnf), 'mnist_exps_gn_new_corruption')

DEEPTH: 6, WIDTH: 2048, LCP: 0.2


/home/mara/venv/local/lib/python2.7/site-packages/keras/activations.py:115: UserWarning: Do not pass a layer instance (such as Activation) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
/home/mara/venv/lib/python2.7/site-packages/ipykernel_launcher.py:134: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("fl...)`


Train on 54000 samples, validate on 6000 samples
Epoch 1/300
54000/54000 [==============================] - 12s 228us/step - loss: 1.3563 - acc: 0.6712 - val_loss: 1.0510 - val_acc: 0.7698
Epoch 2/300
54000/54000 [==============================] - 11s 205us/step - loss: 1.0569 - acc: 0.7654 - val_loss: 0.9814 - val_acc: 0.7907
Epoch 3/300
54000/54000 [==============================] - 11s 206us/step - loss: 1.0020 - acc: 0.7799 - val_loss: 0.9541 - val_acc: 0.8002
Epoch 4/300
54000/54000 [==============================] - 11s 200us/step - loss: 0.9687 - acc: 0.7873 - val_loss: 0.9462 - val_acc: 0.7992
Epoch 5/300
54000/54000 [==============================] - 11s 200us/step - loss: 0.9445 - acc: 0.7929 - val_loss: 0.9429 - val_acc: 0.7995
Epoch 6/300
54000/54000 [==============================] - 11s 195us/step - loss: 0.9247 - acc: 0.7976 - val_loss: 0.9420 - val_acc: 0.8000
Epoch 7/300
54000/54000 [==============================] - 11s 197us/step - loss: 0.9066 - acc: 0.8019 - val_lo

Epoch 59/300
54000/54000 [==============================] - 10s 194us/step - loss: 0.0447 - acc: 0.9848 - val_loss: 2.5740 - val_acc: 0.7580
Epoch 60/300
54000/54000 [==============================] - 11s 195us/step - loss: 0.0278 - acc: 0.9913 - val_loss: 2.5491 - val_acc: 0.7622
Epoch 61/300
54000/54000 [==============================] - 11s 194us/step - loss: 0.0141 - acc: 0.9956 - val_loss: 2.6021 - val_acc: 0.7657
Epoch 62/300
54000/54000 [==============================] - 10s 192us/step - loss: 0.0119 - acc: 0.9964 - val_loss: 2.6166 - val_acc: 0.7620
Epoch 63/300
54000/54000 [==============================] - 10s 194us/step - loss: 0.0227 - acc: 0.9930 - val_loss: 2.6073 - val_acc: 0.7628
Epoch 64/300
54000/54000 [==============================] - 11s 196us/step - loss: 0.0286 - acc: 0.9909 - val_loss: 2.6345 - val_acc: 0.7538
Epoch 65/300
54000/54000 [==============================] - 10s 194us/step - loss: 0.0364 - acc: 0.9876 - val_loss: 2.6045 - val_acc: 0.7353
Epoch 66/300


Epoch 116/300
54000/54000 [==============================] - 10s 183us/step - loss: 4.3716e-05 - acc: 1.0000 - val_loss: 2.8898 - val_acc: 0.7728
Epoch 117/300
54000/54000 [==============================] - 10s 178us/step - loss: 4.3034e-05 - acc: 1.0000 - val_loss: 2.8910 - val_acc: 0.7728
Epoch 118/300
54000/54000 [==============================] - 10s 182us/step - loss: 4.2241e-05 - acc: 1.0000 - val_loss: 2.8926 - val_acc: 0.7730
Epoch 119/300
54000/54000 [==============================] - 10s 179us/step - loss: 4.1535e-05 - acc: 1.0000 - val_loss: 2.8928 - val_acc: 0.7728
Epoch 120/300
54000/54000 [==============================] - 10s 181us/step - loss: 4.0897e-05 - acc: 1.0000 - val_loss: 2.8928 - val_acc: 0.7730
Epoch 121/300
54000/54000 [==============================] - 10s 181us/step - loss: 4.0241e-05 - acc: 1.0000 - val_loss: 2.8940 - val_acc: 0.7732
Epoch 122/300
54000/54000 [==============================] - 10s 181us/step - loss: 3.9613e-05 - acc: 1.0000 - val_loss: 2.8

54000/54000 [==============================] - 10s 189us/step - loss: 1.5022e-05 - acc: 1.0000 - val_loss: 2.9424 - val_acc: 0.7728
Epoch 229/300
54000/54000 [==============================] - 10s 191us/step - loss: 1.4935e-05 - acc: 1.0000 - val_loss: 2.9426 - val_acc: 0.7727
Epoch 230/300
54000/54000 [==============================] - 10s 193us/step - loss: 1.4850e-05 - acc: 1.0000 - val_loss: 2.9427 - val_acc: 0.7727
Epoch 231/300
54000/54000 [==============================] - 10s 192us/step - loss: 1.4751e-05 - acc: 1.0000 - val_loss: 2.9427 - val_acc: 0.7728
Epoch 232/300
54000/54000 [==============================] - 10s 190us/step - loss: 1.4677e-05 - acc: 1.0000 - val_loss: 2.9435 - val_acc: 0.7728
Epoch 233/300
54000/54000 [==============================] - 10s 192us/step - loss: 1.4602e-05 - acc: 1.0000 - val_loss: 2.9435 - val_acc: 0.7727
Epoch 234/300
54000/54000 [==============================] - 10s 187us/step - loss: 1.4514e-05 - acc: 1.0000 - val_loss: 2.9437 - val_acc:

Epoch 41/300
54000/54000 [==============================] - 10s 189us/step - loss: 0.2025 - acc: 0.9344 - val_loss: 3.7615 - val_acc: 0.5200
Epoch 42/300
54000/54000 [==============================] - 10s 193us/step - loss: 0.1885 - acc: 0.9387 - val_loss: 4.0643 - val_acc: 0.4518
Epoch 43/300
54000/54000 [==============================] - 10s 187us/step - loss: 0.1839 - acc: 0.9410 - val_loss: 4.4804 - val_acc: 0.4180
Epoch 44/300
54000/54000 [==============================] - 10s 193us/step - loss: 0.1803 - acc: 0.9410 - val_loss: 4.0949 - val_acc: 0.4322
Epoch 45/300
54000/54000 [==============================] - 10s 188us/step - loss: 0.1710 - acc: 0.9435 - val_loss: 3.9947 - val_acc: 0.5123
Epoch 46/300
54000/54000 [==============================] - 10s 186us/step - loss: 0.1602 - acc: 0.9466 - val_loss: 4.1065 - val_acc: 0.4950
Epoch 47/300
54000/54000 [==============================] - 10s 188us/step - loss: 0.1244 - acc: 0.9592 - val_loss: 4.1448 - val_acc: 0.4813
Epoch 48/300


KeyboardInterrupt: 

In [ ]:
mnist = memorization_utils.MNISTRandom()
depths = [6]
#depths = [6,]
#widths = [4,16,64,256, 1024, 2048]
widths = [2048]

#label_corrupt_pbs = [0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
label_corrupt_pbs = [ 0.2, 0.4, 0.6, 0.8, 1.0]
gaussian_noise_fs = [0., 0.2, 0.4, 0.6, 0.8, 1.0]
try:
    del mlp
except:
    pass

for d in depths:
    for w in widths:
        for lcp in label_corrupt_pbs[:]:
            try:
                del mlp
            except:
                pass
            try:
                del mnist
            except:
                pass
            print 'DEEPTH: {}, WIDTH: {}, LCP: {}'.format(d,w,lcp)
            mnist = memorization_utils.MNISTRandom(label_corrupt_p = lcp)
            mlp = memorization_utils.MLP(deep=d, wide=w,epochs=300)
            mlp.train(mnist)
            mlp.save('mnist_{}d_{}w_{}lcp'.format(d,w,lcp), 'mnist_exps_new_corruption')

for d in depths:
    for w in widths:
        for gnf in gaussian_noise_fs:
            try:
                del mlp
            except:
                pass
            try:
                del mnist
            except:
                pass
            print 'DEEPTH: {}, WIDTH: {}, GNF: {}'.format(d,w,gnf)
            mnist = memorization_utils.MNISTRandom(gaussian_noise_f=gnf)
            mlp = memorization_utils.MLP(deep=d, wide=w,epochs=300)
            mlp.train(mnist)
            mlp.save('mnist_{}d_{}w_{}gnf'.format(d,w,gnf), 'mnist_exps_gn_new_corruption')
